In [1]:
!pip install requests beautifulsoup4 boto3 feedparser

In [2]:
!pip install boto3==1.26.135 botocore==1.29.135

In [3]:
import boto3
from botocore.client import Config

ACCESS_KEY = '00539bb0223e8090000000002'
SECRET_KEY = 'K005sOMxLEeQgJNBBqbFg6d5aW99x4w'
ENDPOINT_URL = 'https://s3.us-east-005.backblazeb2.com'
BUCKET_NAME = 'logistics-data'

s3 = boto3.client(
    service_name='s3',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    endpoint_url=ENDPOINT_URL,
    config=Config(signature_version='s3v4')
)

print("✅ Connected to Backblaze B2 with correct boto3 version!")

✅ Connected to Backblaze B2 with correct boto3 version!


In [4]:
import requests
from bs4 import BeautifulSoup

def crawl_static_site(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup.prettify()
    else:
        print(f"❌ Failed to crawl {url}, Status code: {response.status_code}")
        return None

# Crawl Freightwaves homepage
url = "https://www.freightwaves.com/"
html_content = crawl_static_site(url)

if html_content:
    print("✅ Successfully crawled Freightwaves!")

✅ Successfully crawled Freightwaves!


In [5]:
from datetime import datetime

def upload_html_to_b2(source, html_content):
    today = datetime.utcnow().strftime('%Y-%m-%d')
    key = f"{source}/{today}/page.html"

    s3.put_object(
        Bucket=BUCKET_NAME,
        Key=key,
        Body=html_content,
        ContentType='text/html'
    )
    print(f"✅ Uploaded HTML to: {key}")

# Upload Freightwaves homepage HTML
if html_content:
    upload_html_to_b2("freightwaves", html_content)

✅ Uploaded HTML to: freightwaves/2025-04-26/page.html


In [6]:
import feedparser
import json

def crawl_rss_and_upload(rss_url, source_name):
    feed = feedparser.parse(rss_url)
    today = datetime.utcnow().strftime('%Y-%m-%d')

    for entry in feed.entries[:5]:  # Save first 5 articles
        article = {
            "title": entry.title,
            "link": entry.link,
            "published": entry.published
        }

        safe_title = entry.title.replace(" ", "_").replace("/", "_")  # Clean file name
        key = f"{source_name}/{today}/{safe_title}.json"

        s3.put_object(
            Bucket=BUCKET_NAME,
            Key=key,
            Body=json.dumps(article),
            ContentType='application/json'
        )
        print(f"✅ Uploaded News Article: {entry.title}")

# Example: Crawl Freightwaves News RSS
rss_url = "https://www.freightwaves.com/feed"
crawl_rss_and_upload(rss_url, "freightwaves-news")


✅ Uploaded News Article: Saia’s shares sag 30% as tariffs tank demand, exacerbate growing pains
✅ Uploaded News Article: Drewry: Global container volumes to drop 1% on Trump tariffs
✅ Uploaded News Article: DHL upgrades next-day Medical Express service for Latin America
✅ Uploaded News Article: FMCSA denies truck driver learner’s permits for 17-year-olds
✅ Uploaded News Article: S&P Global goes deeper into supply chain with Orbcomm tie-up
